In [1]:
# stdlib
import os

# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [2]:
use_live_bigquery = False

os.environ["TEST_BIGQUERY_APIS_LIVE"] = str(use_live_bigquery)

# Setup

In [3]:
# stdlib
import os

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

'python'

In [4]:
# third party
from apis import make_submit_query
from apis import make_test_query
from helpers import create_user
from helpers import make_user

# syft absolute
import syft as sy
from syft import test_settings

Using Mock API Code, this will query BigQuery. $TEST_BIGQUERY_APIS_LIVE==False


In [5]:
SERVER_PORT = "8080"
SERVER_URL = f"http://localhost:{SERVER_PORT}"

In [6]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port=SERVER_PORT,
    n_consumers=4,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

Autoreload enabled
Starting bigquery-high server on 0.0.0.0:8080
Found `reset=True` in the launch configuration. Resetting the server...


INFO:     Will watch for changes in these directories: ['/home/teo/OpenMined/PySyft/packages/syft/src/syft']
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:     Started reloader process [279259] using WatchFiles


Waiting for server to start..WARN: private key is based on server name: bigquery-high in dev_mode. Don't run this in production.
.

INFO:     Started server process [279273]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


.INFO:     127.0.0.1:35994 - "GET /api/v2/metadata HTTP/1.1" 200 OK
 Done.


SyftInfo: You have launched a development server at http://0.0.0.0:8080.It is intended only for local use.

In [7]:
high_client = sy.login(
    url=SERVER_URL, email="info@openmined.org", password="changethis"
)

INFO:     127.0.0.1:36026 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:36026 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:36026 - "GET /api/v2/api?verify_key=1e5ec44a179d21e0a0721ea7a670b9875d3946c8604f4623903fcafafe03c8e7&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:36036 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <bigquery-high: High side Datasite> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

In [8]:
num_users = 2
users = []
email_disable_index = 0
for i in range(num_users):
    user = make_user()
    create_user(high_client, user)
    user.client = high_client
    if email_disable_index == i:
        user.email_disabled = True
    users.append(user)

INFO:     127.0.0.1:36042 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:36052 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:36026 - "POST /api/v2/register HTTP/1.1" 200 OK
INFO:     127.0.0.1:36026 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:36026 - "GET /api/v2/api?verify_key=999584860adbb384af8cdd0e00935f16e6baa4d89be51dc801069c402624b740&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:36086 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <bigquery-high: High side Datasite> as <nicole-kennedy-fake@openmined.org>
INFO:     127.0.0.1:36092 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:36098 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:36112 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:36026 - "POST /api/v2/register HTTP/1.1" 200 OK
INFO:     127.0.0.1:36026 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:36026 - "GET /api/v2/api?verify_key=d944e2c3fefce65f69

In [9]:
mock_func = make_test_query(
    settings={
        "rate_limiter_enabled": True,
        "calls_per_min": 10,
    }
)

private_func = make_test_query(
    settings={
        "rate_limiter_enabled": False,
    }
)

new_endpoint = sy.TwinAPIEndpoint(
    path="bigquery.test_query",
    description="This endpoint allows to query Bigquery storage via SQL queries.",
    private_function=private_func,
    mock_function=mock_func,
    worker_pool="default-pool",
)

high_client.custom_api.add(endpoint=new_endpoint)

INFO:     127.0.0.1:36150 - "POST /api/v2/api_call HTTP/1.1" 200 OK


SyftSuccess: Endpoint successfully created.

In [10]:
dataset_1 = test_settings.get("dataset_1", default="dataset_1")
dataset_2 = test_settings.get("dataset_2", default="dataset_2")
table_1 = test_settings.get("table_1", default="table_1")
table_2 = test_settings.get("table_2", default="table_2")
table_1_col_id = test_settings.get("table_1_col_id", default="table_id")
table_1_col_score = test_settings.get("table_1_col_score", default="colname")
table_2_col_id = test_settings.get("table_2_col_id", default="table_id")
table_2_col_score = test_settings.get("table_2_col_score", default="colname")

In [11]:
submit_query_function = make_submit_query(
    settings={},
    worker_pool="default-pool",
)

high_client.custom_api.add(endpoint=submit_query_function)

INFO:     127.0.0.1:36154 - "POST /api/v2/api_call HTTP/1.1" 200 OK


SyftSuccess: Endpoint successfully created.

# Create jobs

In [12]:
# third party
from job_helpers import TestJob
from job_helpers import create_job_functions
from job_helpers import create_jobs
from job_helpers import extract_code_path

# Test queries

In [13]:
# stdlib
from typing import Any


def test_query(client: Any, query: str, should_succeed: bool) -> tuple:
    if should_succeed:
        mock_result = client.api.services.bigquery.test_query.mock(sql_query=query)
        private_result = client.api.services.bigquery.test_query.mock(sql_query=query)
    else:
        try:
            mock_result = client.api.services.bigquery.test_query.mock(sql_query=query)
            assert False, f"query should raise, got {mock_result}"
        except Exception as e:
            mock_result = e

        try:
            private_result = client.api.services.bigquery.test_query.private(
                sql_query=query
            )
            assert False, f"query should raise, got {private_result}"
        except Exception as e:
            private_result = e

    return mock_result, private_result

In [14]:
# Loop through all test queries, test if they behave as expected


results = []

for fn in create_job_functions:
    job = fn(users[0])

    print(f"Querying {job.job_type}, should succeed: {job.should_succeed}")
    mock_result, private_result = test_query(high_client, job.query, job.should_succeed)
    results.append((mock_result, private_result))

Querying simple_query, should succeed: True
INFO:     127.0.0.1:36026 - "GET /api/v2/api?verify_key=1e5ec44a179d21e0a0721ea7a670b9875d3946c8604f4623903fcafafe03c8e7&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:36184 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:49444 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Querying wrong_asset_query, should succeed: False
INFO:     127.0.0.1:37836 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:37974 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Querying wrong_syntax_query, should succeed: False


ERROR:syft.service.queue.queue:Unhandled error in handle_message_multiprocessing
Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/api/api.py", line 566, in exec_code
    result = eval(evil_string, _globals, locals())  # nosec
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
  File "mock_test_query", line 19, in mock_test_query
    raise BadRequest(f'{bad_post} Syntax error: Unexpected identifier "{sql_query}" at [1:1]')
google.api_core.exceptions.BadRequest: 400 BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/project-id/queries?prettyPrint=false:  Syntax error: Unexpected identifier "SELECT * FROM table INCORRECT SYNTAX" at [1:1]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/queue/queue.py", line 208, in handle_message_multiprocessing
    result 

SyftError: An error was raised during the execution of the API endpoint call: 
 400 BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/project-id/queries?prettyPrint=false:  Syntax error: Unexpected identifier "SELECT * FROM table INCORRECT SYNTAX" at [1:1]
SyftError: An error was raised during the execution of the API endpoint call: 
 400 BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/project-id/queries?prettyPrint=false:  Syntax error: Unexpected identifier "SELECT * FROM table INCORRECT SYNTAX" at [1:1]
INFO:     127.0.0.1:35802 - "POST /api/v2/api_call HTTP/1.1" 200 OK


ERROR:syft.service.queue.queue:Unhandled error in handle_message_multiprocessing
Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/api/api.py", line 566, in exec_code
    result = eval(evil_string, _globals, locals())  # nosec
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
  File "mock_test_query", line 19, in mock_test_query
    raise BadRequest(f'{bad_post} Syntax error: Unexpected identifier "{sql_query}" at [1:1]')
google.api_core.exceptions.BadRequest: 400 BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/project-id/queries?prettyPrint=false:  Syntax error: Unexpected identifier "SELECT * FROM table INCORRECT SYNTAX" at [1:1]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/queue/queue.py", line 208, in handle_message_multiprocessing
    result 

SyftError: An error was raised during the execution of the API endpoint call: 
 400 BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/project-id/queries?prettyPrint=false:  Syntax error: Unexpected identifier "SELECT * FROM table INCORRECT SYNTAX" at [1:1]
SyftError: An error was raised during the execution of the API endpoint call: 
 400 BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/project-id/queries?prettyPrint=false:  Syntax error: Unexpected identifier "SELECT * FROM table INCORRECT SYNTAX" at [1:1]
INFO:     127.0.0.1:35908 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Querying job_too_much_text, should succeed: False


...

SyftError: An error was raised during the execution of the API endpoint call: 
 400 BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/project-id/queries?prettyPrint=false:  Syntax error: Unexpected identifier "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

ERROR:syft.service.queue.queue:Unhandled error in handle_message_multiprocessing
Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/api/api.py", line 566, in exec_code
    result = eval(evil_string, _globals, locals())  # nosec
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
  File "mock_test_query", line 19, in mock_test_query
    raise BadRequest(f'{bad_post} Syntax error: Unexpected identifier "{sql_query}" at [1:1]')
google.api_core.exceptions.BadRequest: 400 BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/project-id/queries?prettyPrint=false:  Syntax error: Unexpected identifier "SELECT table_id, AVG(colname) AS average_score
    FROM dataset_2.table_2
    GROUP BY table_id
    LIMIT 469472<script>alert('XSS in query')</script>" at [1:1]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/teo/OpenMi

SyftError: An error was raised during the execution of the API endpoint call: 
 400 BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/project-id/queries?prettyPrint=false:  Syntax error: Unexpected identifier "SELECT table_id, AVG(colname) AS average_score
    FROM dataset_2.table_2
    GROUP BY table_id
    LIMIT 469472<script>alert('XSS in query')</script>" at [1:1]
SyftError: An error was raised during the execution of the API endpoint call: 
 400 BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/project-id/queries?prettyPrint=false:  Syntax error: Unexpected identifier "SELECT table_id, AVG(colname) AS average_score
    FROM dataset_2.table_2
    GROUP BY table_id
    LIMIT 469472<script>alert('XSS in query')</script>" at [1:1]
INFO:     127.0.0.1:46952 - "POST /api/v2/api_call HTTP/1.1" 200 OK


ERROR:syft.service.queue.queue:Unhandled error in handle_message_multiprocessing
Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/api/api.py", line 566, in exec_code
    result = eval(evil_string, _globals, locals())  # nosec
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
  File "mock_test_query", line 19, in mock_test_query
    raise BadRequest(f'{bad_post} Syntax error: Unexpected identifier "{sql_query}" at [1:1]')
google.api_core.exceptions.BadRequest: 400 BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/project-id/queries?prettyPrint=false:  Syntax error: Unexpected identifier "SELECT table_id, AVG(colname) AS average_score
    FROM dataset_2.table_2
    GROUP BY table_id
    LIMIT 469472<script>alert('XSS in query')</script>" at [1:1]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/teo/OpenMi

SyftError: An error was raised during the execution of the API endpoint call: 
 400 BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/project-id/queries?prettyPrint=false:  Syntax error: Unexpected identifier "SELECT table_id, AVG(colname) AS average_score
    FROM dataset_2.table_2
    GROUP BY table_id
    LIMIT 469472<script>alert('XSS in query')</script>" at [1:1]
SyftError: An error was raised during the execution of the API endpoint call: 
 400 BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/project-id/queries?prettyPrint=false:  Syntax error: Unexpected identifier "SELECT table_id, AVG(colname) AS average_score
    FROM dataset_2.table_2
    GROUP BY table_id
    LIMIT 469472<script>alert('XSS in query')</script>" at [1:1]
INFO:     127.0.0.1:47162 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Querying job_many_columns, should succeed: True
INFO:     127.0.0.1:57846 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:44498 

In [15]:
n_per_user = 10

jobs = create_jobs(users, n_per_user=n_per_user)

print(f"num jobs: {len(jobs)}")
for job in jobs[:2]:
    print(f"Job type: {job.job_type}, should succeed: {job.should_succeed}")

assert len(jobs) == len(users) * n_per_user
assert all(isinstance(j, TestJob) for j in jobs)
assert all(job.client is not None for job in jobs)

num jobs: 10
Job type: wrong_asset_query, should succeed: False
Job type: job_query_xss, should succeed: False


# Submit jobs


In [16]:
# stdlib
import time

responses = []

for job in jobs:
    client = job.client

    if not job.should_submit:
        # Submitting should throw error (eg func_name invalid syntax)
        with sy.raises(sy.SyftException):
            response = client.api.services.bigquery.submit_query(
                func_name=job.func_name, query=job.query
            )
        responses.append(None)
    else:
        response = client.api.services.bigquery.submit_query(
            func_name=job.func_name, query=job.query
        )
        job.code_path = extract_code_path(response)
        responses.append(response)

    time.sleep(1)

INFO:     127.0.0.1:44094 - "GET /api/v2/api?verify_key=999584860adbb384af8cdd0e00935f16e6baa4d89be51dc801069c402624b740&communication_protocol=dev HTTP/1.1" 200 OK
SyftSuccess: Syft function 'execute_query' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.
INFO:     127.0.0.1:44120 - "POST /api/v2/api_call HTTP/1.1" 200 OK
SyftSuccess: Syft function 'execute_query' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.
INFO:     127.0.0.1:53146 - "POST /api/v2/api_call HTTP/1.1" 200 OK
SyftSuccess: Syft function 'execute_query' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.
INFO:     127.0.0.1:51696 - "POST /api/v2/api_call HTTP/1.1" 200 OK


ERROR:syft.service.queue.queue:Unhandled error in handle_message_multiprocessing
Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/utils.py", line 63, in parse_code
    tree = ast.parse(raw_code)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/teo/anaconda3/envs/syft_3.12/lib/python3.12/ast.py", line 52, in parse
    return compile(source, filename, mode, flags,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<unknown>", line 2
    def job_funcname_xss_464420<script>alert('XSS in funcname')</script>(query: str, endpoint):
                               ^
SyntaxError: expected '('

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1300, in decorator
    parse_user_code(
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1368, in parse_user_cod

SyftError: Ops something went wrong during this endpoint execution, please contact your admin.
SyftError: Ops something went wrong during this endpoint execution, please contact your admin.
INFO:     127.0.0.1:41380 - "POST /api/v2/api_call HTTP/1.1" 200 OK
SyftSuccess: Syft function 'execute_query' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.
INFO:     127.0.0.1:34420 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:40216 - "GET /api/v2/api?verify_key=d944e2c3fefce65f693108e9648a4899e3325557609be2703254bb4631200a7d&communication_protocol=dev HTTP/1.1" 200 OK


ERROR:syft.service.queue.queue:Unhandled error in handle_message_multiprocessing
Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/utils.py", line 63, in parse_code
    tree = ast.parse(raw_code)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/teo/anaconda3/envs/syft_3.12/lib/python3.12/ast.py", line 52, in parse
    return compile(source, filename, mode, flags,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<unknown>", line 2
    def job_funcname_xss_2b061f<script>alert('XSS in funcname')</script>(query: str, endpoint):
                               ^
SyntaxError: expected '('

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1300, in decorator
    parse_user_code(
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1368, in parse_user_cod

SyftError: Ops something went wrong during this endpoint execution, please contact your admin.
SyftError: Ops something went wrong during this endpoint execution, please contact your admin.
INFO:     127.0.0.1:40218 - "POST /api/v2/api_call HTTP/1.1" 200 OK
SyftSuccess: Syft function 'execute_query' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.
INFO:     127.0.0.1:36334 - "POST /api/v2/api_call HTTP/1.1" 200 OK
SyftSuccess: Syft function 'execute_query' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.
INFO:     127.0.0.1:54146 - "POST /api/v2/api_call HTTP/1.1" 200 OK
SyftSuccess: Syft function 'execute_query' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.
INFO:     127.0

In [17]:
for job in jobs:
    print(
        f"Job {job.func_name:.20} is submitted, {job.should_submit}, should be submitted {job.is_submitted}"
    )

assert all(job.is_submitted == job.should_submit for job in jobs)

Job wrong_asset_query_d7 is submitted, True, should be submitted True
Job job_query_xss_9211fd is submitted, True, should be submitted True
Job job_too_much_text_aa is submitted, True, should be submitted True
Job job_funcname_xss_464 is submitted, False, should be submitted False
Job wrong_syntax_query_0 is submitted, True, should be submitted True
Job job_funcname_xss_2b0 is submitted, False, should be submitted False
Job wrong_syntax_query_9 is submitted, True, should be submitted True
Job wrong_syntax_query_f is submitted, True, should be submitted True
Job wrong_syntax_query_6 is submitted, True, should be submitted True
Job wrong_asset_query_6f is submitted, True, should be submitted True


## Test: cannot execute

In [18]:
submitted_jobs = [job for job in jobs if job.should_submit]

assert len(submitted_jobs)  # failsafe for next tests

In [19]:
# Blocking

for job in submitted_jobs:
    execute_code_fn = getattr(job.client.code, job.code_path)
    with sy.raises(
        sy.SyftException(public_message="*Your code is waiting for approval*")
    ):
        result = execute_code_fn()

INFO:     127.0.0.1:60860 - "GET /api/v2/api?verify_key=999584860adbb384af8cdd0e00935f16e6baa4d89be51dc801069c402624b740&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:60884 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:60890 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:60896 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:60900 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:60860 - "GET /api/v2/api?verify_key=d944e2c3fefce65f693108e9648a4899e3325557609be2703254bb4631200a7d&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:60906 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:60910 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:60920 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:60930 - "POST /api/v2/api_call HTTP/1.1" 200 OK


In [20]:
# Nonblocking

# syft absolute
from syft.service.job.job_stash import JobStatus

for job in submitted_jobs:
    execute_code_fn = getattr(job.client.code, job.code_path)
    result_job = execute_code_fn(blocking=False)
    result_job.wait()
    assert isinstance(result_job.result, sy.SyftError)
    assert result_job.status == JobStatus.ERRORED
    time.sleep(1)

INFO:     127.0.0.1:60934 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:60950 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:60956 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:60964 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:60978 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:60998 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:32780 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:32794 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:32800 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:32838 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:32846 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:32876 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:32890 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:32904 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:32914 - "POST /api/v2/api_ca

ERROR:syft.service.queue.queue:Unhandled error in handle_message_multiprocessing
Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/queue/queue.py", line 208, in handle_message_multiprocessing
    result = call_method(context, *queue_item.args, **queue_item.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/service.py", line 485, in _decorator
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/action/action_service.py", line 826, in execute
    ).unwrap()
      ^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/types/result.py", line 89, in unwrap
    raise self.value
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/types/result.py", line 111, in wrapper
    output = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^

SyftError: <class 'syft.service.code.user_code.UserCodeStatusCollection'> Your code is waiting for approval. Code status on server 'bigquery-high' is 'UserCodeStatus.PENDING'.
SyftError: <class 'syft.service.code.user_code.UserCodeStatusCollection'> Your code is waiting for approval. Code status on server 'bigquery-high' is 'UserCodeStatus.PENDING'.
INFO:     127.0.0.1:33028 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:52764 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:52776 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:52782 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:52788 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:52800 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:52812 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:52818 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:52842 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:52844 - "POST /a

ERROR:syft.service.queue.queue:Unhandled error in handle_message_multiprocessing
Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/queue/queue.py", line 208, in handle_message_multiprocessing
    result = call_method(context, *queue_item.args, **queue_item.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/service.py", line 485, in _decorator
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/action/action_service.py", line 826, in execute
    ).unwrap()
      ^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/types/result.py", line 89, in unwrap
    raise self.value
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/types/result.py", line 111, in wrapper
    output = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^

INFO:     127.0.0.1:53040 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50284 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50296 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50298 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50314 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50324 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50366 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50374 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50400 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50404 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50448 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50456 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50466 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50468 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:50492 - "POST /api/v2/api_ca

...

ERROR:syft.service.queue.queue:Unhandled error in handle_message_multiprocessing
Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/queue/queue.py", line 208, in handle_message_multiprocessing
    result = call_method(context, *queue_item.args, **queue_item.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/service.py", line 485, in _decorator
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/action/action_service.py", line 826, in execute
    ).unwrap()
      ^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/types/result.py", line 89, in unwrap
    raise self.value
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/types/result.py", line 111, in wrapper
    output = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^

SyftError: <class 'syft.service.code.user_code.UserCodeStatusCollection'> Your code is waiting for approval. Code status on server 'bigquery-high' is 'UserCodeStatus.PENDING'.
SyftError: <class 'syft.service.code.user_code.UserCodeStatusCollection'> Your code is waiting for approval. Code status on server 'bigquery-high' is 'UserCodeStatus.PENDING'.
INFO:     127.0.0.1:41278 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:41300 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:41316 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:41330 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:41338 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:41340 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:41370 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:41372 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:41386 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:41388 - "POST /a

ERROR:syft.service.queue.queue:Unhandled error in handle_message_multiprocessing
Traceback (most recent call last):
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/queue/queue.py", line 208, in handle_message_multiprocessing
    result = call_method(context, *queue_item.args, **queue_item.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/service.py", line 485, in _decorator
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/service/action/action_service.py", line 826, in execute
    ).unwrap()
      ^^^^^^^^
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/types/result.py", line 89, in unwrap
    raise self.value
  File "/home/teo/OpenMined/PySyft/packages/syft/src/syft/types/result.py", line 111, in wrapper
    output = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^

SyftError: <class 'syft.service.code.user_code.UserCodeStatusCollection'> Your code is waiting for approval. Code status on server 'bigquery-high' is 'UserCodeStatus.PENDING'.
SyftError: <class 'syft.service.code.user_code.UserCodeStatusCollection'> Your code is waiting for approval. Code status on server 'bigquery-high' is 'UserCodeStatus.PENDING'.
INFO:     127.0.0.1:37086 - "POST /api/v2/api_call HTTP/1.1" 200 OK


INFO:     127.0.0.1:56110 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:56114 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:56114 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:56114 - "GET /api/v2/api?verify_key=1e5ec44a179d21e0a0721ea7a670b9875d3946c8604f4623903fcafafe03c8e7&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:56146 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:56156 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:56172 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:56188 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:56190 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:56194 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:56208 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:56218 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:56220 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:5623